In [1]:
from PIL import Image, ExifTags
from PIL.ExifTags import TAGS, GPSTAGS
import datetime

# Extract EXIF Meta

In [2]:
def get_exif_data(image):
    """Returns a dictionary from the exif data of an PIL Image item. Also converts the GPS Tags"""
    exif_data = {}
    info = image._getexif()
    if info:
        for tag, value in info.items():
            decoded = TAGS.get(tag, tag)
            if decoded == "GPSInfo":
                gps_data = {}
                for t in value:
                    sub_decoded = GPSTAGS.get(t, t)
                    gps_data[sub_decoded] = value[t]

                exif_data[decoded] = gps_data
            else:
                exif_data[decoded] = value

    return exif_data

In [3]:
img = Image.open("/Users/snn2spade/Desktop/Pai Trip 29-2 Feb 19/DCIM2/IMG_0852.jpeg")
exif = get_exif_data(img)
del exif["MakerNote"]
exif

{'GPSInfo': {'GPSLatitudeRef': 'N',
  'GPSLatitude': ((19, 1), (18, 1), (1713, 100)),
  'GPSLongitudeRef': 'E',
  'GPSLongitude': ((98, 1), (27, 1), (1001, 100)),
  'GPSAltitudeRef': b'\x00',
  'GPSAltitude': (567, 1),
  'GPSTimeStamp': ((10, 1), (45, 1), (13, 1)),
  'GPSSpeedRef': 'K',
  'GPSSpeed': (6437, 129950),
  'GPSImgDirectionRef': 'T',
  'GPSImgDirection': (241436, 995),
  'GPSDestBearingRef': 'T',
  'GPSDestBearing': (241436, 995),
  'GPSDateStamp': '2019:02:01',
  'GPSHPositioningError': (17201, 2134)},
 'ResolutionUnit': 2,
 'ExifOffset': 210,
 'Make': 'Apple',
 'Model': 'iPhone XS',
 'Software': 'Photos 4.0',
 'Orientation': 1,
 'DateTime': '2019:02:01 17:45:13',
 'YCbCrPositioning': 1,
 'XResolution': (72, 1),
 'YResolution': (72, 1),
 'ExifVersion': b'0221',
 'ComponentsConfiguration': b'\x01\x02\x03\x00',
 'ShutterSpeedValue': (93361, 8727),
 'DateTimeOriginal': '2019:02:01 17:45:13',
 'DateTimeDigitized': '2019:02:01 17:45:13',
 'ApertureValue': (54823, 32325),
 'Brigh

# General Metadata

In [4]:
print("ISO = {}".format(exif["ISOSpeedRatings"]))

ISO = 25


In [5]:
print("Focal Length = {} mm".format(exif["FocalLength"][0]/exif["FocalLength"][1]))

Focal Length = 4.25 mm


In [6]:
print("Model = {}".format(exif["Model"]))

Model = iPhone XS


In [7]:
created_date = datetime.datetime.strptime(exif["DateTimeOriginal"],"%Y:%m:%d %H:%M:%S")
print("Created Date = {}".format(created_date))

Created Date = 2019-02-01 17:45:13


In [8]:
print("Speed shutter = {}".format(str(exif["ExposureTime"][0])+"/"+str(exif["ExposureTime"][1])))

Speed shutter = 1/1661


In [9]:
print("Fnumber = {}".format(exif["FNumber"][0]/exif["FNumber"][1]))

Fnumber = 1.8


# Convert GPS Information

In [10]:
def _get_if_exist(data, key):
    if key in data:
        return data[key]
    return None

def _convert_to_degress(value):
    """Helper function to convert the GPS coordinates stored in the EXIF to degress in float format"""
    d0 = value[0][0]
    d1 = value[0][1]
    d = float(d0) / float(d1)

    m0 = value[1][0]
    m1 = value[1][1]
    m = float(m0) / float(m1)

    s0 = value[2][0]
    s1 = value[2][1]
    s = float(s0) / float(s1)

    return d + (m / 60.0) + (s / 3600.0)

def get_lat_lon(exif_data):
    """Returns the latitude and longitude, if available, from the provided exif_data (obtained through get_exif_data above)"""
    lat = None
    lon = None

    if "GPSInfo" in exif_data:		
        gps_info = exif_data["GPSInfo"]

        gps_latitude = _get_if_exist(gps_info, "GPSLatitude")
        gps_latitude_ref = _get_if_exist(gps_info, 'GPSLatitudeRef')
        gps_longitude = _get_if_exist(gps_info, 'GPSLongitude')
        gps_longitude_ref = _get_if_exist(gps_info, 'GPSLongitudeRef')

        if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
            lat = _convert_to_degress(gps_latitude)
            if gps_latitude_ref != "N":                     
                lat = 0 - lat

            lon = _convert_to_degress(gps_longitude)
            if gps_longitude_ref != "E":
                lon = 0 - lon

    return lat, lon

In [18]:
exif["GPSLocation"] = round(get_lat_lon(exif)[0],6) , round(get_lat_lon(exif)[1],6)
print("GPSLocation = {}".format(exif["GPSLocation"]))

GPSLocation = (19.304758, 98.452781)


In [19]:
exif["GPSHPositioningError"] = round(exif["GPSInfo"]["GPSHPositioningError"][0]/exif["GPSInfo"]["GPSHPositioningError"][1])
print("GPSHPositioningError = {} m".format(exif["GPSHPositioningError"]))

GPSHPositioningError = 8 m


In [23]:
exif["GoogleMapURL"] = "https://maps.google.com/?q={},{}".format(exif["GPSLocation"][0],exif["GPSLocation"][1])
print("GoogleMapURL = '{}'".format(exif["GoogleMapURL"]))

GoogleMapURL = 'https://maps.google.com/?q=19.304758,98.452781'
